# Filtering Annotations

In [3]:
%load_ext autoreload
%autoreload 2

import json
import os
import sys
sys.path.insert(0, '/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset')
import header as head

annotation_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/annotations/en-scheme-annotations.json"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Extracting all charlie hebdo annotations

In [ ]:
head.extract_annotations("charliehebdo",annotation_path)

Organize the annotations by source tweets with their corresponding Direct Replies and Deep Replies.

In [ ]:
# Organize all the annotation based on source tweets
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/filtered_annotations.json") as f:
    data = json.load(f)

head.org_annotations(data)

For each source tweet, we extract all direct replies and deep replies which the respondents follow the user that they replied to.

In [ ]:
# Filter out reactions from followers from organized annotation
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/annotations/organized_annotations.json") as f:
    org_annots = json.load(f)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/react_follows.json") as f:
    react_follow = json.load(f)

for annot in list(org_annots):
    try: 
        reacts = react_follow[annot]
    
        for rep in (org_annots[annot])["Direct Replies"]:
            tweetId = rep["tweetid"]
            if not (tweetId in reacts): (org_annots[annot])["Direct Replies"].pop(rep)
        
        for rep in (org_annots[annot])["Deep Replies"]:
            tweetId = rep["tweetid"]
            if not (tweetId in reacts): (org_annots[annot])["Deep Replies"].pop(rep)
    
    except: 
        org_annots.pop(annot)

head.writeToJSON(head.getRoot(),"follower_annotations",org_annots)

We translate the follower_annotation data to refrence each source tweet by its userId instead of tweetId. 

In [ ]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/Id-conversions/tweet_to_user.json") as f:
    tweet_to_user = json.load(f)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/annotations/follower_annotations.json") as f:
    follower_annotations = json.load(f)

dic = {}
for annot in follower_annotations:
    src_userId = tweet_to_user[annot]
    src_tweet = follower_annotations[annot]
    dic[src_userId] = {
        "source_tweetId" : annot,
        "support" : (src_tweet["Source Tweets"])["support"],
        "certainty" : (src_tweet["Source Tweets"])["certainty"],
        "supportList" : {
            "certain" : [],
            "somewhat-certain" : [],
            "uncertain" : []
        },
        "denyList": {
            "certain" : [],
            "somewhat-certain" : [],
            "uncertain" : []
        },
        "commentList" : []
    }
    cur_dic = dic[src_userId]
    dir_rep = src_tweet["Direct Replies"]
    for d in dir_rep:
        id = tweet_to_user[d["tweetid"]]
        if d["responsetype-vs-source"] == "comment":
            cur_dic["commentList"].append(id)

        elif d["responsetype-vs-source"] == "agreed":
            supporList = cur_dic["supportList"]
            if d["certainty"] == "certain":
                supporList["certain"].append(id)
            elif d["certainty"] == "somewhat-certain":
                supporList["somewhat-certain"].append(id)
            else :
                supporList["uncertain"].append(id)
        else:
            denyList = cur_dic["denyList"]
            if d["certainty"] == "certain":
                denyList["certain"].append(id)
            elif d["certainty"] == "somewhat-certain":
                denyList["somewhat-certain"].append(id)
            else :
                denyList["uncertain"].append(id)


head.writeToJSON("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/annotations","userId_annotations",dic)

In [ ]:
# Further organize "filtered_annotation" file by using their tweetIds as keys.
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/annotations/filtered_annotations.json") as f:
    annots = json.load(f)

soruces = annots["Source Tweets"]
dir_rep = annots["Direct Replies"]
deep_rep = annots["Deep Replies"]

dic = {}
dic["Source_Tweets"] = {}
dic["Reply_Tweets"] = {}
src_dic = dic["Source_Tweets"]
rep_dic = dic["Reply_Tweets"]

for s in soruces:
    support = s["support"]
    if support != "underspecified":
        if s["certainty"] != "uncertain":
            src_dic[s["tweetid"]] = {
                "support": s["support"],
                "certainty" : s["certainty"]
            }


for d in dir_rep:
    respons_type = d["responsetype-vs-source"]
    if (respons_type != "comment" and respons_type != "appeal-for-more-information"):
        if d["certainty"] != "uncertain" and d["certainty"] != "underspecified":
            rep_dic[d["tweetid"]] = {
                "threadid" : d["threadid"],
                "responsetype-vs-source": d["responsetype-vs-source"],
                "certainty" : d["certainty"]
            }
        
for d in deep_rep:
    respons_type = d["responsetype-vs-source"]
    if (respons_type != "comment" and respons_type != "appeal-for-more-information"):
        if d["certainty"] != "uncertain" and d["certainty"] != "underspecified":
            rep_dic[d["tweetid"]] = {
                "threadid" : d["threadid"],
                "responsetype-vs-source": d["responsetype-vs-source"],
                "certainty" : d["certainty"]
            }

head.writeToJSON(annotations_path,"tweetId_annotations",dic)